<a href="https://colab.research.google.com/github/harshitadd/DP-NLP/blob/main/BERTNewsClassificationDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install opacus[dev] --quiet

     |████████████████████████████████| 102kB 2.8MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 21.6MB 1.5MB/s 
     |████████████████████████████████| 112kB 50.7MB/s 
     |████████████████████████████████| 143kB 57.9MB/s 
     |████████████████████████████████| 4.9MB 18.5MB/s 
     |████████████████████████████████| 368kB 39.1MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 747kB 28.6MB/s 
     |████████████████████████████████| 727kB 36.6MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: sphinx-autodoc-typehints 1.12.0 has requirement Sphinx>=3.0, but you'll have sphinx 1.8.5 which is incompa

In [1]:
!pip install transformers --quiet

     |████████████████████████████████| 2.3MB 5.4MB/s 
     |████████████████████████████████| 3.3MB 32.4MB/s 
     |████████████████████████████████| 901kB 26.9MB/s 


In [3]:
import zipfile
import urllib.request
import os
import torch
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import transformers
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from opacus.utils.uniform_sampler import UniformWithReplacementSampler
from opacus import PrivacyEngine
from torch.utils.data import TensorDataset
from transformers.data.processors.utils import InputExample
from transformers.data.processors.glue import glue_convert_examples_to_features
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification


In [4]:
DATA_DIR = "/content/"

In [ ]:
def download_and_extract(data_dir):
    print("Extracting Train zip...")
    filename = "train.csv.zip"
    with zipfile.ZipFile(filename) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(filename)
    print("Completed!")

    print("Extracting Test zip...")
    filename = "test.csv.zip"
    with zipfile.ZipFile(filename) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(filename)
    print("Completed!")

download_and_extract(DATA_DIR)

In [ ]:
train_path =  '/content/train.csv'
dev_path = '/content/test.csv'

df_train = pd.read_csv(train_path)
df_test = pd.read_csv(dev_path)

In [ ]:
df_train.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [ ]:
df_train = df_train.drop('Title', axis = 1)

In [ ]:
df_test = df_test.drop('Title', axis = 1)

In [ ]:
df_train.columns

Index(['Class Index', 'Description'], dtype='object')

In [ ]:
df_test.columns

Index(['Class Index', 'Description'], dtype='object')

In [ ]:
# 1-World, 2-Sports, 3-Business, 4-Sci/Tech

In [ ]:
df_train = df_train.replace({'Class Index': {1: 'World', 2: 'Sports', 3: 'Business', 4: 'Sci/Tech'}})

In [ ]:
df_test = df_test.replace({'Class Index': {1: 'World', 2: 'Sports', 3: 'Business', 4: 'Sci/Tech'}})

In [ ]:
df_test

,Class Index,Description
0,Business,Unions representing workers at Turner Newall...
1,Sci/Tech,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,Sci/Tech,AP - A company founded by a chemistry research...
3,Sci/Tech,AP - It's barely dawn when Mike Fitzpatrick st...
4,Sci/Tech,AP - Southern California's smog-fighting agenc...
...,...,...
7595,World,Ukrainian presidential candidate Viktor Yushch...
7596,Sports,With the supply of attractive pitching options...
7597,Sports,Like Roger Clemens did almost exactly eight ye...
7598,Business,SINGAPORE : Doctors in the United States have ...


In [ ]:
model_name = "bert-base-cased"
config = BertConfig.from_pretrained(
    model_name,
    num_labels=4,
)
tokenizer = BertTokenizer.from_pretrained(
    "bert-base-cased",
    do_lower_case=False,
)
model = BertForSequenceClassification.from_pretrained(
    "bert-base-cased",
    config=config,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
trainable_layers = [model.bert.encoder.layer[-1], model.bert.pooler, model.classifier]
total_params = 0
trainable_params = 0

for p in model.parameters():
        p.requires_grad = False
        total_params += p.numel()

for layer in trainable_layers:
    for p in layer.parameters():
        p.requires_grad = True
        trainable_params += p.numel()

print(f"Total parameters count: {total_params}") # ~108M
print(f"Trainable parameters count: {trainable_params}") # ~7M

Total parameters count: 108313348
Trainable parameters count: 7681540


In [ ]:
################################################################################
LABEL_LIST = ['World','Sports','Business','Sci/Tech'] 
MAX_SEQ_LENGHT = 128


def _create_examples(df, set_type):
    """ Convert raw dataframe to a list of InputExample. Filter malformed examples
    """
    examples = []
    for index, row in df.iterrows():
        # if row['Category'] in LABEL_LIST:
        #     continue
        # if isinstance(row['ArticleId'], str) or isinstance(row['Text'], str):
        #     continue
            
        guid = f"{index}-{set_type}"
        examples.append(
            InputExample(guid=guid, text_a=row['Description'], label=row['Class Index']))
    return examples

def _df_to_features(df, set_type):
    """ Pre-process text. This method will:
    1) tokenize inputs
    2) cut or pad each sequence to MAX_SEQ_LENGHT
    3) convert tokens into ids
    
    The output will contain:
    `input_ids` - padded token ids sequence
    `attention mask` - mask indicating padded tokens
    `token_type_ids` - mask indicating the split between premise and hypothesis
    `label` - label
    """
    examples = _create_examples(df, set_type)
    
    #backward compatibility with older transformers versions
    legacy_kwards = {}
    from packaging import version
    if version.parse(transformers.__version__) < version.parse("2.9.0"):
        legacy_kwards = {
            "pad_on_left": False,
            "pad_token": tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
            "pad_token_segment_id": 0,
        }
    
    return glue_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        label_list=LABEL_LIST,
        max_length=MAX_SEQ_LENGHT,
        output_mode="classification",
        **legacy_kwards,
    )

def _features_to_dataset(features):
    """ Convert features from `_df_to_features` into a single dataset
    """
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor(
        [f.attention_mask for f in features], dtype=torch.long
    )
    all_token_type_ids = torch.tensor(
        [f.token_type_ids for f in features], dtype=torch.long
    )
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    dataset = TensorDataset(
        all_input_ids, all_attention_mask, all_token_type_ids, all_labels
    )

    return dataset

In [ ]:
train_features = _df_to_features(df_train, "train")

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [ ]:
test_features = _df_to_features(df_test, "test")

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


In [ ]:
train_dataset = _features_to_dataset(train_features)
test_dataset = _features_to_dataset(test_features)

In [ ]:
BATCH_SIZE = 8
VIRTUAL_BATCH_SIZE = 8
assert VIRTUAL_BATCH_SIZE % BATCH_SIZE == 0 # VIRTUAL_BATCH_SIZE should be divisible by BATCH_SIZE
N_ACCUMULATION_STEPS = int(VIRTUAL_BATCH_SIZE / BATCH_SIZE)

In [ ]:

SAMPLE_RATE = BATCH_SIZE / len(train_dataset)

train_sampler=UniformWithReplacementSampler(
    num_samples=len(train_dataset),
    sample_rate=SAMPLE_RATE,
)
train_dataloader = DataLoader(train_dataset, batch_sampler=train_sampler)

test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
# Move the model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the model to train mode (HuggingFace models load in eval mode)
model = model.train()
# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, eps=1e-8)

In [ ]:
print(torch.__version__)

1.8.1+cu101


In [ ]:
EPOCHS = 1
LOGGING_INTERVAL = 1000 # once every how many steps we run evaluation cycle and report metrics
EPSILON = 7.5
DELTA = 1 / len(train_dataloader) # Parameter for privacy accounting. Probability of not achieving privacy guarantees

In [ ]:

def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):    
    model.eval()

    loss_arr = []
    accuracy_arr = []
    
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2],
                      'labels':         batch[3]}

            outputs = model(**inputs)
            loss, logits = outputs[:2]
            
            preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
            labels = inputs['labels'].detach().cpu().numpy()
            
            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))
    
    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

In [ ]:
MAX_GRAD_NORM = 0.1

privacy_engine = PrivacyEngine(
    module=model,
    sample_rate=SAMPLE_RATE * N_ACCUMULATION_STEPS,
    target_delta = DELTA,
    target_epsilon = EPSILON, 
    epochs = EPOCHS,
    max_grad_norm=MAX_GRAD_NORM,
)
privacy_engine.attach(optimizer)

/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:523: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:195: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


In [ ]:
for epoch in range(1, EPOCHS+1):
    losses = []
    for step, batch in enumerate(tqdm(train_dataloader)):
        
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'labels':         batch[3]}
        outputs = model(**inputs) # output = loss, logits, hidden_states, attentions
        
        loss = outputs[0]
        loss.backward()
        
        losses.append(loss.item())

        # We process small batches of size BATCH_SIZE, 
        # until they're accumulated to a batch of size VIRTUAL_BATCH_SIZE.
        # Only then we make a real `.step()` and update model weights
        if (step + 1) % N_ACCUMULATION_STEPS == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
        else:
            optimizer.virtual_step()

        if step > 0 and step % LOGGING_INTERVAL == 0:
            train_loss = np.mean(losses)
            eps, alpha = optimizer.privacy_engine.get_privacy_spent(DELTA)

            eval_loss, eval_accuracy = evaluate(model)

            print(
                f"Epoch: {epoch} | "
                f"Step: {step} | "
                f"Train loss: {train_loss:.3f} | "
                f"Eval loss: {eval_loss:.3f} | "
                f"Eval accuracy: {eval_accuracy:.3f} | "
                f"ɛ: {eps:.2f} (α: {alpha})"
            )

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


tensor([1, 0, 0, 0, 0, 1, 3, 3], device='cuda:0')
tensor([2, 2, 1, 2, 3, 3, 2, 0], device='cuda:0')
tensor([0, 2, 3, 2, 2], device='cuda:0')
tensor([0, 1, 2, 2, 1, 0, 0, 1], device='cuda:0')
tensor([0, 1, 2, 3, 2, 1, 2, 1, 0, 3, 2, 0], device='cuda:0')


RuntimeError: ignored